# Pruning with our automatic structured Pruning framework
Welcome to an end-to-end example for magnitude-based weight pruning

**Summary**

In this tutorial, you will:

* Train a tf.keras model for CIFAR10 from scratch.
* Fine tune the model by applying the pruning Framework and see the accuracy.

If you want to execute this notebook in Google Colab, uncomment the code below.

In [1]:
# !git clone https://github.com/Hahn-Schickard/Automatic-Structured-Pruning
# !echo $CWD
# !cp -rf /content/Automatic-Structured-Pruning/* /content/

In [2]:
import pruning

## Train a model for CIFAR10 without pruning
Download and prepare the CIFAR10 dataset.
The CIFAR10 dataset contains 60,000 color images in 10 classes, with 6,000 images in each class. The dataset is divided into 50,000 training images and 10,000 testing images. The classes are mutually exclusive and there is no overlap between them.

Create the convolutional base
The 6 lines of code below define the convolutional base using a common pattern: a stack of Conv2D and MaxPooling2D layers.

As input, a CNN takes tensors of shape (image_height, image_width, color_channels), ignoring the batch size. If you are new to these dimensions, color_channels refers to (R,G,B). In this example, you will configure our CNN to process inputs of shape (32, 32, 3), which is the format of CIFAR images. You can do this by passing the argument input_shape to our first layer.

To complete our model, you will feed the last output tensor from the convolutional base (of shape (4, 4, 64)) into one or more Dense layers to perform classification. Dense layers take vectors as input (which are 1D), while the current output is a 3D tensor. First, you will flatten (or unroll) the 3D output to 1D, then add one or more Dense layers on top. CIFAR has 10 output classes, so you use a final Dense layer with 10 outputs and a softmax activation.

In [3]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models


(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0

Above, you can see that the output of every Conv2D and MaxPooling2D layer is a 3D tensor of shape (height, width, channels). The width and height dimensions tend to shrink as you go deeper in the network. The number of output channels for each Conv2D layer is controlled by the first argument (e.g., 32 or 64). Typically, as the width and height shrink, you can afford (computationally) to add more output channels in each Conv2D layer.
As you can see, our (4, 4, 64) outputs were flattened into vectors of shape (1024) before going through two Dense layers.

## Compile and train the model

In [4]:

comp = {
"optimizer":'adam',
"loss": tf.keras.losses.SparseCategoricalCrossentropy(),
"metrics": ['accuracy']}

model.compile(**comp)
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]

model.fit(train_images, train_labels, validation_split=0.2, epochs=30, batch_size=128, callbacks=callbacks)

test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)

Epoch 1/30
313/313 [==============================] - 13s 9ms/step - loss: 1.9751 - accuracy: 0.2511 - val_loss: 1.6634 - val_accuracy: 0.3810
Epoch 2/30
313/313 [==============================] - 2s 7ms/step - loss: 1.6420 - accuracy: 0.3889 - val_loss: 1.5236 - val_accuracy: 0.4487
Epoch 3/30
313/313 [==============================] - 2s 6ms/step - loss: 1.4921 - accuracy: 0.4547 - val_loss: 1.3377 - val_accuracy: 0.5095
Epoch 4/30
313/313 [==============================] - 2s 6ms/step - loss: 1.3850 - accuracy: 0.4965 - val_loss: 1.3040 - val_accuracy: 0.5307
Epoch 5/30
313/313 [==============================] - 2s 6ms/step - loss: 1.3070 - accuracy: 0.5298 - val_loss: 1.2000 - val_accuracy: 0.5697
Epoch 6/30
313/313 [==============================] - 2s 6ms/step - loss: 1.2501 - accuracy: 0.5571 - val_loss: 1.1762 - val_accuracy: 0.5813
Epoch 7/30
313/313 [==============================] - 2s 6ms/step - loss: 1.2175 - accuracy: 0.5702 - val_loss: 1.1173 - val_accuracy: 0.6057
Epoch

In [5]:
model.save('CIFAR10_model.h5') 

## Fine-tune pre-trained model with pruning
You will apply pruning to the whole model and see this in the model summary.

In this example, you prune the model with 30% dense pruning and 40% filter pruning.

In [6]:
dense_prune_rate=30
conv_prune_rate=40
prunemodel=pruning.prune_model('./CIFAR10_model.h5', dense_prune_rate, conv_prune_rate,'L2', num_classes=10)

Finish with pruning
Before pruning:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
flatten (Flatten)   

c:\Users\dk100\Documents\Automatic-Structured-Pruning\pruning_helper_functions.py:108: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(layer_types), np.array(layer_params), \


We see how we get less parameter in the pruned model.

## Compile and re-train the model

In [7]:
prunemodel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 20)        560       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 20)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 39)        7059      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 39)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 39)          13728     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 39)          0         
_________________________________________________________________
flatten (Flatten)            (None, 156)               0

In [8]:
comp = {
"optimizer":'adam',
"loss": tf.keras.losses.SparseCategoricalCrossentropy(),
"metrics": ['accuracy']}

prunemodel.compile(**comp)

prunemodel.fit(train_images, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
1250/1250 [==============================] - 7s 5ms/step - loss: 1.4468 - accuracy: 0.4859 - val_loss: 1.1077 - val_accuracy: 0.6084
Epoch 2/10
1250/1250 [==============================] - 7s 5ms/step - loss: 1.1553 - accuracy: 0.6024 - val_loss: 1.0277 - val_accuracy: 0.6404
Epoch 3/10
1250/1250 [==============================] - 7s 6ms/step - loss: 1.0625 - accuracy: 0.6363 - val_loss: 0.9939 - val_accuracy: 0.6495
Epoch 4/10
1250/1250 [==============================] - 7s 6ms/step - loss: 1.0076 - accuracy: 0.6567 - val_loss: 0.9646 - val_accuracy: 0.6618
Epoch 5/10
1250/1250 [==============================] - 6s 5ms/step - loss: 0.9676 - accuracy: 0.6732 - val_loss: 0.9892 - val_accuracy: 0.6569
Epoch 6/10
1250/1250 [==============================] - 6s 5ms/step - loss: 0.9372 - accuracy: 0.6801 - val_loss: 0.9426 - val_accuracy: 0.6759
Epoch 7/10
1250/1250 [==============================] - 7s 5ms/step - loss: 0.9104 - accuracy: 0.6924 - val_loss: 1.0318 - val_accuracy:

Compare both models

In [9]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
test_loss, test_acc = prunemodel.evaluate(test_images,  test_labels, verbose=2)

313/313 - 1s - loss: 0.9178 - accuracy: 0.6950
313/313 - 1s - loss: 0.9234 - accuracy: 0.6941


In [10]:
print(f"Total number of parameters before pruning: {model.count_params()}")
print(f"Total number of parameters after pruning: {prunemodel.count_params()}")
print(f"Pruned model contains only {(prunemodel.count_params()/model.count_params())*100:.2f} % of the original number of parameters.")

Total number of parameters before pruning: 75178
Total number of parameters after pruning: 29710
Pruned model contains only 39.52 % of the original number of parameters.


# Prune a model to a maximum accuracy loss

We define the arguments to compile the model. In this case, we only want to have an accuracy loss of 3%.
In this example we have loaded the data directly from a TensorFlow dataset. Therefore, we do not have a defined dataloader (path or file). However, the structure of the training data is the same as the data we would read from a Python file. Therefore, we use a trick here and pass an existing FILE from the current folder as the dataloader path. This way the correct functions will be executed afterwards and no error will be issued.

In [11]:
comp = {
  "optimizer": 'adam',
  "loss": tf.keras.losses.SparseCategoricalCrossentropy(),
  "metrics": 'accuracy'
}

auto_model = pruning.pruning_for_acc('./CIFAR10_model.h5', train_images, train_labels, comp, pruning_acc=None,
                                  max_acc_loss=5, num_classes=10, label_one_hot=False)

313/313 [==============================] - 1s 3ms/step - loss: 0.6162 - accuracy: 0.7899
Start model accuracy: 78.9900004863739%
Next pruning factors: 5
Finish with pruning
Before pruning:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________

In [12]:
auto_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0

Compare both models

In [13]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
test_loss, test_acc = auto_model.evaluate(test_images,  test_labels, verbose=2)

313/313 - 1s - loss: 0.9178 - accuracy: 0.6950
313/313 - 1s - loss: 0.9178 - accuracy: 0.6950


In [14]:
print(f"Total number of parameters before pruning: {model.count_params()}")
print(f"Total number of parameters after pruning: {auto_model.count_params()}")
print(f"Pruned model contains only {(auto_model.count_params()/model.count_params())*100:.2f} % of the original number of parameters.")

Total number of parameters before pruning: 75178
Total number of parameters after pruning: 75178
Pruned model contains only 100.00 % of the original number of parameters.
